# Geospatial Cyclists

In [1]:
print('hello, world!')

hello, world!


In [2]:
import pandas as pd
import geopandas as gpd
import folium
from ipyleaflet import *                       
from ipywidgets import *     
from IPython.display import display

Final = False

In [13]:
Location = (-36.85, 174.76)
ZOOM = 4

base = Map(center=Location, zoom=13, control_scale=True,
           basemap=basemaps.CartoDB.Positron)

if Final:
    base.layout.width = '1125px'
    base.layout.height = '2436px'
    
else:
    base.layout.width = '750px'
    base.layout.height = '1624px'

def handle_map_interaction(**kwargs):
   # markerU = CircleMarker(location=(0,0), draggable=True, visible = False)
   # m.add_layer(markerU)
    #m.add_layer(markerU)
    #print(kwargs)

    if kwargs['type'] == 'click':

        NLoc =  kwargs['coordinates']
        
        print(NLoc)

base.on_interaction(handle_map_interaction)

selDf = gpd.read_file('./Data/SA2_Trimmed/1.shp')


geoDf = selDf.to_crs(4326)
lonCent = (geoDf.bounds.maxx + geoDf.bounds.minx).mean()/2
latCent = (geoDf.bounds.maxy + geoDf.bounds.miny).mean()/2

base.center = (latCent,lonCent)
geoData = GeoData(geo_dataframe=geoDf, name='NZ SA2 FW')

base.add_layer(geoData)


control = LayersControl(position='topright')
base.add_control(control)


base

Map(center=[-36.89933304214278, 174.7803547802036], controls=(ZoomControl(options=['position', 'zoom_in_text',…

In [10]:
import numpy as np

DepData = np.genfromtxt('./Data/SA2D.csv', skip_header= 1, delimiter = ',',dtype = str)

DepData

array([['100100', 'North Cape', '10', '1134', '1602'],
       ['100200', 'Rangaunu Harbour', '9', '1105', '2310'],
       ['100300', 'Inlets Far North District', '7', '1031', '45'],
       ...,
       ['400014', 'Oceanic Antipodes Islands', '', '', '0'],
       ['400015', 'Antipodes Islands', '', '', '0'],
       ['400016', 'Ross Dependency', '', '', '0']], dtype='<U44')

In [11]:
base.list_features

AttributeError: 'Map' object has no attribute 'list_features'

In [ ]:
# potential layers to add
# pavement condition
# traffic flow

In [ ]:
fitness = widgets.Dropdown(
            options=['I cycle all the time', 'I cycle every now and then', "I'm new to cycling, go easy!"],
            disabled=False,
            layout={'width': '30%'})
cafe = widgets.Checkbox(
    value=False,
    description='Include Cafe',
    indent=False
)
water = widgets.Checkbox(
    value=False,
    description='Include Water Fountain',
    indent=False
)
submit = widgets.Button(description='Submit')

display(VBox([fitness, water, cafe, submit]))

In [ ]:
'''
MAP 2: Deprivation Index by SA2
-get deprivation index data from postgis
-create choropleth map
-add popups to show underlying data (parsed as geojson)
'''
# sql query to get table created above
sql = 'SELECT * FROM chch_depindex_sa2'

# add table as geodataframe, remove nulls if available (shouldn't be, this step may be redundant but have had issues, so have left in)
depindex_gdf = gpd.GeoDataFrame.from_postgis(sql, engine, geom_col='geometry', crs=CRS)
depindex_gdf.dropna(how='any', axis=0, inplace=True)

# create folium map, centre based on constants
depindex_m = folium.Map(location=CHCH, zoom_start=DEFAULT_ZOOM, tiles=None, control_scale=True)

# add choropleth layer to map
depindex_m.choropleth(geo_data=depindex_gdf,                                     # geometry df
                      name='Degrees of Deprivation',                             # layer name
                      data=depindex_gdf,                                         # data df
                      columns=['sa2_code_2018', 'depindex_score_2018'],          # columns of interest
                      key_on='feature.properties.sa2_code_2018',                 # key to join geom with data
                      fill_color='RdPu',                                         # colour gradient
                      fill_opacity=0.8,                                          # make slightly transparent
                      line_opacity=0.2,                                          # make lines thinly transparent
                      legend_name='Degree of Socioeconomic Deprivation (2018)',  # name on legend
                      smooth_factor=0)                                           # smoothing of polylines

# style and highlight functions
# changes fill colour and properties when selected, serializable dictionary for both
style_function = lambda x: {'fillColor': '#ffffff', 'color':'#000000', 'fillOpacity': 0.1, 'weight': 0.1}
highlight_function = lambda x: {'fillColor': '#000000', 'color':'#000000', 'fillOpacity': 0.50, 'weight': 0.1}

# create popup from folium features added in choropleth. use style and highlight functions above
# use data from geojson parsing of geodataframe to get other table information when sa2 area is highlighted
popup_info = folium.features.GeoJson(
            depindex_gdf, 
            style_function=style_function, 
            control=False,
            highlight_function=highlight_function, 
            tooltip=folium.features.GeoJsonTooltip(
                fields=['sa2_code_2018', 'sa2_name', 'depindex_score_2018'],
                aliases=['Meshblock ID: ', 'Name: ', 'Deprivation Index Score: '],
                style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;")))

# add popup info as child to map, keep in front of other elements
depindex_m.add_child(popup_info)
depindex_m.keep_in_front(popup_info)

# add basemap, layer control
folium.TileLayer('stamenterrain').add_to(depindex_m)
folium.LayerControl().add_to(depindex_m)

# display map
depindex_m